## División de cursos mediante Algoritmos Genéticos

 **Contexto:**
En una escuela secundaria, 60 alumnos pasaron exitosamente el curso de ingreso.Ahora deben ser asignados a 3 cursos de 20 alumnos cada uno. Cada alumno eligió dos compañeros con quienes le gustaría estar en el mismo curso.


**Objetivo:**
Construir una asignacion que maximice la utilidad total del grupo.

**Reglas:**
- Cada curso debe tener exactamente 20 alumnos.
- Cada alumno debe pertenecer a un único curso.

**Definiciones de utilidad (fitness):**
- +1 punto por cada amigo elegido que este en el mismo curso (Osea + 2 si los 2 amigos están en el mismo curso)
- -3 puntos si los dos amigos terminan juntos en otro curso.
- -1 punto si el alumno y sus dos amigos terminan todos en cursos distintos.

In [208]:
# Listado de alumnos y sus preferencias
alumnos_amigos = {
    'Ana': ['Bruno', 'Camila'], 'Bruno': ['Ana', 'Diego'], 'Camila': ['Bruno', 'Elena'],
    'Diego': ['Camila', 'Francisco'], 'Elena': ['Diego', 'Gabriela'], 'Francisco': ['Elena', 'Hugo'],
    'Gabriela': ['Francisco', 'Isabel'], 'Hugo': ['Gabriela', 'Juan'], 'Isabel': ['Hugo', 'Karla'],
    'Juan': ['Isabel', 'Leonardo'], 'Karla': ['Juan', 'Mariana'], 'Leonardo': ['Karla', 'Nicolas'],
    'Mariana': ['Leonardo', 'Olivia'], 'Nicolas': ['Mariana', 'Pablo'], 'Olivia': ['Nicolas', 'Raul'],
    'Pablo': ['Olivia', 'Sofia'], 'Raul': ['Pablo', 'Tomas'], 'Sofia': ['Raul', 'Ursula'],
    'Tomas': ['Sofia', 'Valeria'], 'Ursula': ['Tomas', 'Walter'], 'Valeria': ['Ursula', 'Xavier'],
    'Walter': ['Valeria', 'Yara'], 'Xavier': ['Walter', 'Zoe'], 'Yara': ['Xavier', 'Andres'],
    'Zoe': ['Yara', 'Belen'], 'Andres': ['Zoe', 'Carlos'], 'Belen': ['Andres', 'Daniela'],
    'Carlos': ['Belen', 'Esteban'], 'Daniela': ['Carlos', 'Fernanda'], 'Esteban': ['Daniela', 'Gustavo'],
    'Fernanda': ['Esteban', 'Helena'], 'Gustavo': ['Fernanda', 'Ignacio'], 'Helena': ['Gustavo', 'Julieta'],
    'Ignacio': ['Helena', 'Kevin'], 'Julieta': ['Ignacio', 'Laura'], 'Kevin': ['Julieta', 'Mateo'],
    'Laura': ['Kevin', 'Natalia'], 'Mateo': ['Laura', 'Octavio'], 'Natalia': ['Mateo', 'Paula'],
    'Octavio': ['Natalia', 'Ricardo'], 'Paula': ['Octavio', 'Sabrina'], 'Ricardo': ['Paula', 'Tadeo'],
    'Sabrina': ['Ricardo', 'Ursula'], 'Tadeo': ['Sabrina', 'Victor'], 'Victor': ['Ursula', 'Ximena'],
    'Wanda': ['Victor', 'Yamila'], 'Ximena': ['Wanda', 'Zoe'], 'Yamila': ['Ximena', 'Ana'], 'Zoe': ['Yamila', 'Bruno'],
    'Beltran': ['Ana', 'Diego'], 'Catalina': ['Francisco', 'Gabriela'], 'Dario': ['Hugo', 'Isabel'],
    'Emilia': ['Leonardo', 'Mariana'], 'Fabian': ['Nicolas', 'Olivia'], 'Gisela': ['Pablo', 'Raul'],
    'Hernan': ['Sofia', 'Valeria'], 'Isidoro': ['Walter', 'Xavier'], 'Jimena': ['Yara', 'Andres'],
    'Horacio': ['Belen', 'Carlos'], 'Luz': ['Daniela', 'Esteban'], 'Martin': ['Fernanda', 'Gustavo']
}

**Entregables:**
- 1. Código en Python con comentarios.
- 2. Breve descripción sobre cómo definieron el genoma, la funcion de aptitud, el tamaño de la población inicial, la tasa de mutación y por qué eligieron un determinado método de selección de padres.
- 3. División de cursos que maximiza la utilidad total y valor de esa utilidad total.

**Bonus 1 (opcional):**
Implementar elitismo:
- Guardar el mejor individuo en cada generación y pasarlo directamente sin modificaciones.

**Bonus 2 (opcional):**
Por motivos personales los alumnos Xavier y Zoe deben compartir curso si o si con sus dos amigos (Walter y Zoe y Yara y Belen). Modifica el código para garantizar esto.

In [209]:
import random
from copy import deepcopy

### **Paso 1: Configuro los parametros del AG**

In [210]:
alumnos = list(alumnos_amigos.keys()) # convierte el diccionario en una lista de nombres.
cantidad_alumnos = len(alumnos) # cuenta cuántos alumnos hay.
cantidad_cursos = 3 # vamos a dividir a los alumnos en 3 cursos.

In [211]:
# Capacidad por curso (distribución equitativa con resto)
# Distribuye equitativamente a los alumnos entre los 3 cursos. Ejemplo: si hay 60 alumnos, cada curso tendrá 20.
base = cantidad_alumnos // cantidad_cursos
resto = cantidad_alumnos % cantidad_cursos
capacidades = [base + (1 if i < resto else 0) for i in range(cantidad_cursos)]

In [212]:
# Parámetros del algoritmo genético
population_size = 100 # cuántas soluciones posibles vamos a mantener en cada generación.
generations = 500 # cuántas veces repetimos el proceso para mejorar soluciones.
mutation_rate = 0.1 # probabilidad de hacer un pequeño cambio aleatorio en una solución.
tournament_size = 3 # cuántas soluciones se comparan en cada “torneo” para elegir padres.
elitism = True # siempre conservamos la mejor solución.

In [213]:
# Alumnos que deben estar en el mismo curso sí o sí
# fixed_block dice a qué curso va cada uno de ellos. Se guardan sus posiciones para usarlas más adelante.
forced_block = ['Xavier', 'Walter', 'Zoe', 'Yara', 'Belen']
block_indices = [alumnos.index(name) for name in forced_block]

### **Paso 2: Create Population**

In [214]:
# Crea una solución inicial aleatoria: asigna a cada alumno a uno de los 3 cursos.
# Respeta el bloque de alumnos fijos y se asegura de no superar la capacidad de cada curso.
# El resultado es una lista donde cada posición representa un alumno y su valor es su curso asignado.

In [215]:
def create_individual():
    """Crea un individuo válido respetando el bloque forzado y capacidades."""
    cursos = capacidades.copy()
    block_course = random.randint(0, cantidad_cursos - 1)
    cursos[block_course] -= len(block_indices)
    
    individual = [None] * cantidad_alumnos
    for idx in block_indices:
        individual[idx] = block_course

    restantes = []
    for c in range(cantidad_cursos):
        restantes += [c] * cursos[c]
    random.shuffle(restantes)

    j = 0
    for i in range(cantidad_alumnos):
        if individual[i] is None:
            individual[i] = restantes[j]
            j += 1
    return individual

### **Paso 3: Fitness Function**

In [216]:
# Mide qué tan buena es una solución. Recorre cada alumno y verifica:

# +1 punto por cada amigo que está en el mismo curso.
# –3 puntos si sus dos amigos están juntos pero sin él.
# –1 punto si todos están separados.

# Devuelve un número: cuanto más alto, mejor.

In [217]:
def fitness(individual):
    """Calcula la utilidad total según las reglas de amistad y penalizaciones."""
    total = 0
    for i, alumno in enumerate(alumnos):
        curso = individual[i]
        amigo1, amigo2 = alumnos_amigos[alumno]
        i1, i2 = alumnos.index(amigo1), alumnos.index(amigo2)
        c1, c2 = individual[i1], individual[i2]
        if c1 == curso: total += 1
        if c2 == curso: total += 1
        if c1 == c2 and c1 != curso: total -= 3
        if curso != c1 and curso != c2 and c1 != c2: total -= 1
    return total

### **Paso 4: Parent Selection**

In [218]:
def tournament_selection(population, fitnesses):
    """Selecciona al mejor individuo de un torneo de k aleatorios."""
    participants = random.sample(list(zip(population, fitnesses)), tournament_size)
    participants.sort(key=lambda x: x[1], reverse=True)
    return deepcopy(participants[0][0])

## 🏆 Selección por Torneo (Tournament Selection)

Este método se usa para elegir **quién va a reproducirse** (es decir, quién va a ser “padre” o “madre” de la próxima generación).

---

### 🔍 ¿Cómo funciona paso a paso?

1. Elegimos al azar `k` individuos de la población (por ejemplo: `k = 3`).

Imaginá que tenés una población con estos individuos y sus valores de fitness:

| Individuo | Fitness |
|-----------|---------|
| A         | 12      |
| B         | 40      |
| C         | 25      |
| D         | 60      |
| E         | 33      |

Supongamos que elegís **3 individuos al azar** → B, D, A

2. Comparamos sus valores de fitness:

| Seleccionado | Fitness |
|--------------|---------|
| B            | 40      |
| D            | 60 ✅ mejor |
| A            | 12      |

3. El mejor del torneo gana.

En este caso, el mejor es **D** (fitness 60), entonces se selecciona como padre.

---

### 🧠 ¿Por qué usar este método?

#### ✅ Ventajas:
- Evita elegir solo al mejor siempre, dando oportunidad a otros.
- Controla la presión selectiva: si aumentás `k`, ganan más seguido los mejores.
- No necesita normalizar los valores (como el método del dardo o accept-reject).

#### ⚖️ Presión Selectiva:
- `k` chico → más aleatoriedad (más diversidad)
- `k` grande → más elitismo (menos diversidad)

Es un **buen balance entre exploración y explotación**.


### **Paso 5: Cruza (crossover) + reparación del bloque y capacidades**

In [219]:
# Crea un hijo mezclando genes de los padres (cada posición puede venir de padre1 o padre2).
# Luego usa repair() para corregir si algún curso tiene más alumnos de los que debería.
# También fuerza que los alumnos fijos estén en el curso que les corresponde.

In [220]:
def crossover(parent1, parent2):
    """Cruza uniformemente dos padres y repara el resultado."""
    child = [parent1[i] if random.random() < 0.5 else parent2[i] for i in range(cantidad_alumnos)]
    block_course = child[block_indices[0]]
    for idx in block_indices:
        child[idx] = block_course
    return repair(child)

def repair(child):
    """Ajusta para que cada curso tenga la cantidad justa de alumnos."""
    conteos = [child.count(c) for c in range(cantidad_cursos)]
    for c in range(cantidad_cursos):
        while conteos[c] > capacidades[c]:
            idxs = [i for i in range(cantidad_alumnos) if child[i] == c and i not in block_indices]
            idx = random.choice(idxs)
            for d in range(cantidad_cursos):
                if conteos[d] < capacidades[d]:
                    child[idx] = d
                    conteos[c] -= 1
                    conteos[d] += 1
                    break
    return child

### **Paso 6: Mutation**

In [221]:
def mutate(child):
    """Intercambia dos posiciones aleatorias (que no pertenezcan al bloque)."""
    if random.random() < mutation_rate:
        i, j = random.sample([x for x in range(cantidad_alumnos) if x not in block_indices], 2)
        child[i], child[j] = child[j], child[i]
    return child

### **Paso 7: Replacement + Main Loop**

In [222]:
# En el loop principal se incluye la opción elitism = True y se copia el mejor individuo a la nueva población sin modificaciones -- BONUS 1

In [223]:
# Inicializar población
population = [create_individual() for _ in range(population_size)]
fitnesses = [fitness(ind) for ind in population]

# Bucle principal de evolución
for _ in range(generations):
    new_population = []
    if elitism:
        best_idx = max(range(population_size), key=lambda i: fitnesses[i])
        new_population.append(deepcopy(population[best_idx]))
    while len(new_population) < population_size:
        p1 = tournament_selection(population, fitnesses)
        p2 = tournament_selection(population, fitnesses)
        child = crossover(p1, p2)
        child = mutate(child)
        new_population.append(child)
    population = new_population
    fitnesses = [fitness(ind) for ind in population]

### **Paso 8: Muestro el Resultado Final**

In [224]:
best = population[fitnesses.index(max(fitnesses))]
best_value = max(fitnesses)

print(f"Utilidad total mejor encontrada: {best_value}\n")
for c in range(cantidad_cursos):
    grupo = [alumnos[i] for i in range(cantidad_alumnos) if best[i] == c]
    print(f"Curso {c+1} ({len(grupo)} alumnos):")
    print(grupo, "\n")

Utilidad total mejor encontrada: 94

Curso 1 (20 alumnos):
['Elena', 'Francisco', 'Gabriela', 'Hugo', 'Isabel', 'Sofia', 'Tomas', 'Ursula', 'Valeria', 'Kevin', 'Laura', 'Mateo', 'Natalia', 'Victor', 'Wanda', 'Ximena', 'Yamila', 'Catalina', 'Dario', 'Hernan'] 

Curso 2 (20 alumnos):
['Juan', 'Karla', 'Leonardo', 'Mariana', 'Nicolas', 'Olivia', 'Pablo', 'Raul', 'Daniela', 'Esteban', 'Fernanda', 'Gustavo', 'Helena', 'Ignacio', 'Julieta', 'Emilia', 'Fabian', 'Gisela', 'Luz', 'Martin'] 

Curso 3 (20 alumnos):
['Ana', 'Bruno', 'Camila', 'Diego', 'Walter', 'Xavier', 'Yara', 'Zoe', 'Andres', 'Belen', 'Carlos', 'Octavio', 'Paula', 'Ricardo', 'Sabrina', 'Tadeo', 'Beltran', 'Isidoro', 'Jimena', 'Horacio'] 

